<a id='sect0'></a>
## <font color='darkblue'>Preface</font>
雖然我年紀已經不小, 但是追朔 [FP (Functional programming) 的歷史](https://en.wikipedia.org/wiki/Functional_programming#History), 我也只能算年輕:
> The lambda calculus, developed in the 1930s by Alonzo Church, is a formal system of computation built from function application. 

<br/><br/>


<b><font size='3ptx'>既然歷史攸久, 因此一言難盡</font></b>, 在這邊只會 吹吹皮毛, 希望至少可以 cover 下面的內容:
* <font size='3ptx'><b><a href='#sect1'>Basic FP terminology (Function/Side effect/Closure ...)</a></b></font>
* <font size='3ptx'><b><a href='#sect2'>Lambda 用法 & 範例</a></b></font>
* <font size='3ptx'><b><a href='#sect3'>常常和 Lambda 一起使用的其他函數 map/filter/functools.reduce</a></b></font>
* <font size='3ptx'><b><a href='#sect4'>可以接受的 Lambda 使用時機</a></b></font>
* <font size='3ptx'><b><a href='#sect5'>Review 時會被打槍的用法</a></b></font>
* <font size='3ptx'><b><a href='#sect6'>FPU 簡介</a></b></font>

In [9]:
#!pip install fpu

In [10]:
from fpu.flist import *
from functools import partial
from typing import Sequence
from collections.abc import Iterable

<a id='sect1'></a>
## <font color='darkblue'>Basic FP terminology</font>
* <font size='3ptx'><b><a href='#sect1_1'>FP Terminology - Imperative vs Declarative</a></b></font>
* <font size='3ptx'><b><a href='#sect1_2'>FP Terminology - Closure</a></b></font>
* <font size='3ptx'><b><a href='#sect1_3'>FP Terminology - Currying</a></b></font>
<br/>

Functional programming has a [long history](https://en.wikipedia.org/wiki/Functional_programming#History). In a nutshell, **its a style of programming where you focus on transforming data through the use of small expressions that ideally don’t contain side effects.** In other words, when you call <font color='blue'>my_fun(1, 2)</font>, it will always return the same result. This is achieved by **immutable data** typical of a functional language.

![function & side effect](images/1.PNG)

([image source](https://www.fpcomplete.com/blog/2017/04/pure-functional-programming/))
<br/>

<a id='sect1_1'></a>
### <font color='darkgreen'>FP Terminology - Imperative vs Declarative</font>
你可以把 `Imperative` 與 `Decleartive` 想成將編程語言分類的一種方法. (<font color='brown'>顏色, 大小, 形狀 etc</font>). 稍後會說明這兩種陣營的語言寫起來的差別:

![function & side effect](images/2.PNG)
<br/>

#### <font size='3ptx'>Imperative</font>
底下是 [Imperative 語言的 wiki 說明](https://en.wikipedia.org/wiki/Imperative_programming):
> **Imperative programming** is like building assembly lines, which take some initial global state as raw material, apply various specific transformations, mutations to it as this material is pushed through the line, and at the end comes off the end product, the final global state, that represents the result of the computation. Each step needs to change, rotate, massage the workpiece precisely one specific way, so that it is prepared for subsequent steps downstream. Every step downstream depend on every previous step, and their order is therefore fixed and rigid. Because of these dependencies, an individual computational step has not much use and meaning in itself, but only in the context of all the others, and to understand it, one must understand how the whole line works.
<br/>

現今大部分的語言都屬於 imperative 陣營, 看描述很辛苦, 透過比對兩者的編程 style 可以很容易發現兩者不同. 底下是 imperative style 的編成方式:

In [7]:
salaries = [
    (True, 9000),   # (Is female, salary)
    (False, 12000), 
    (False, 6000),
    (True, 14000),
]

In [23]:
def imperative_way(salaries):
    '''Gets the sum of salaries of female and male.
    
    Args:
        salaries: List of salary. Each element is of tuple(is_female: bool, salary: int)
    
    Returns:
        Tuple(Sum of female salaries, Sum of male salaries)
    '''
    female_sum = male_sum = 0
    for is_female, salary in salaries:
        if is_female:
            female_sum += salary
        else:
            male_sum += salary
            
    return (female_sum, male_sum)

In [3]:
imperative_way(salaries)

(23000, 18000)

#### <font size='3ptx'>Declarative</font>
底下是 [Declarative 語言的 wiki 說明](https://en.wikipedia.org/wiki/Declarative_programming):
> A style of building the structure and elements of computer programs—that expresses the logic of a computation without describing its control flow.

In [24]:
def add(a, b):
    return a + b

def salary_sum(is_female: bool):
    '''Return calculator to sum up the salary based on female/male
    
    Args:
        is_female: True to return calculator to sum up salaries of female
            False to return calculator to sum up salaries of male.
            
    Returns:
        Calculator to sum up salaries.
    '''
    def _salary_sum(salaries):
        flist = fl(salaries)
        return flist.filter(lambda t: t[0] == is_female) \
                    .map(lambda t: t[1]) \
                    .foldLeft(0, add)
    return _salary_sum

In [10]:
def declarative_way(salaries):
    return (
        salary_sum(is_female=True)(salaries),  # Salary sum of female
        salary_sum(is_female=False)(salaries), # Salary sum of male
    )

In [11]:
declarative_way(salaries)

(23000, 18000)

<a id='sect1_2'></a>
### <font color='darkgreen'>FP Terminology - Closure</font> ([back](#sect1))
A [**Closure**](https://en.wikipedia.org/wiki/Closure_(computer_programming)) is a function which **simply creates a scope that allows the function to access and manipulate the variables in enclosing scopes**. Normally, you will follow below steps to create a Closure in Python:
* We have to create a nested function (a function inside another function).
* This nested function has to refer to a variable defined inside the enclosing function.
* The enclosing function has to return the nested function

<br/>

簡單說就是你的 Function object 綁定一個封閉的 name space (<font color='brown'>[這篇](https://dboyliao.medium.com/%E8%81%8A%E8%81%8A-python-closure-ebd63ff0146f)介紹還蠻清楚, 可以參考</font>), 直接來看範例理解:

In [15]:
def contain_N(n):
    def _inner(sequence: Sequence):
        return n in sequence
    
    return _inner

In [16]:
contain_5 = contain_N(5)
contain_10 = contain_N(10)

In [18]:
my_datas = [1, 2, 3, 4, 5]
print(f'my_data={my_datas} contains 5? {contain_5(my_datas)}')
print(f'my_data={my_datas} contains 10? {contain_10(my_datas)}')

my_data=[1, 2, 3, 4, 5] contains 5? True
my_data=[1, 2, 3, 4, 5] contains 10? False


上面的函數 `contain_N` 返回一個 closure. 該 closure 綁訂了變數 `n`. (<font color='blue'>contain_5</font> <font color='brown'>綁定的 `n` 為 5;</font> <font color='blue'>contain_10</font> <font color='brown'>綁定的 `n` 為 10</font>)

<a id='sect1_3'></a>
### <font color='darkgreen'>FP Terminology - Currying</font> ([back](#sect1))
底下是 wiki 上對 [**currying**](https://en.wikipedia.org/wiki/Curry_(programming_language)) 的說明:
> <b>Currying is like a kind of incremental binding of function arguments</b>. It is the technique of breaking down the evaluation of a function that takes multiple arguments into evaluating a sequence of single-argument functions.

<br/>

很可惜在 Python 預設的函數並不支援這個特性, 幸運的是在模組 [**functools**](https://docs.python.org/3/library/functools.html) 有提供 [partial](https://docs.python.org/3/library/functools.html#functools.partial) 來模擬 currying 的好處. 直接來看範例: 

In [41]:
def sum_salary_by_sex(*args):
    def _sum_salary_by_sex(is_female: bool, salaries: Sequence) -> int:
        return sum(map(lambda t: t[1], filter(lambda t: t[0]==is_female, salaries)))
    
    if len(args) == 1:
        return partial(_sum_salary_by_sex, args[0])
    
    return _sum_salary_by_sex(*args)

In [42]:
# Get female salaries
sum_salary_by_sex(True, salaries)

23000

In [43]:
# # Get female salaries in currying way
sum_salary_by_sex(True)(salaries)

23000

In [39]:
# Get male salaries
sum_salary_by_sex(False, salaries)

18000

In [45]:
# Get male salaries in curring way
sum_salary_by_sex(False)(salaries)

18000

In [46]:
sum_salary_by_female = sum_salary_by_sex(True)
sum_salary_by_male = sum_salary_by_sex(False)

In [47]:
sum_salary_by_female(salaries)

23000

In [48]:
sum_salary_by_male(salaries)

18000

我們透過 currying 的特性便可以得到新的函數 <font color='blue'>sum_salary_by_female</font> 與 <font color='blue'>sum_salary_by_male</font>. 是不是很方便?

<a id='sect2'></a>
## <font color='darkblue'>Lambda 用法 & 範例</font> ([back](#sect0))
[**Lambda**](https://docs.python.org/3/reference/expressions.html#lambda) 在 Python 是一個關鍵字用來定義 匿名函數. 底下是使用 lambda 匿名函數的注意事項:
* It can only contain expressions and can’t include statements ([No Statesments](#sect2_1)) in its body.
* [It is written as a single line of execution](#sect2_2).
* [It does not support type annotations.](#sect2_3)
* [It can be immediately invoked](#sect2_4) ([IIFE](https://en.wikipedia.org/wiki/Immediately_invoked_function_expression)).

<a id='sect2_1'></a>
### <font color='darkgreen'>No Statements</font>
<b><font size='3ptx' color='darkred'>A lambda function can’t contain any statements</font></b>. In a lambda function, statements like `return`, `pass`, `assert`, or `raise` will raise a [**SyntaxError**](https://realpython.com/invalid-syntax-python/) exception. Here’s an example of adding assert to the body of a lambda:
```python
>>> (lambda x: assert x == 2)(2)
  File "<input>", line 1
    (lambda x: assert x == 2)(2)
                    ^
SyntaxError: invalid syntax
```
<br/>

<a id='sect2_2'></a>
### <font color='darkgreen'>Single Expression</font>
<font size='3ptx'><b>In contrast to a normal function, a Python lambda function is a single expression</b></font>. Although, in the body of a lambda, you can spread the expression over several lines using parentheses or a multiline string, it remains a single expression:
```python
>>> (lambda x:
... (x % 2 and 'odd' or 'even'))(3)
'odd'
```
<br/>

The example above returns the string 'odd' when the lambda argument is odd, and 'even' when the argument is even. It spreads across two lines because it is contained in a set of parentheses, but it remains a single expression.

<a id='sect2_3'></a>
### <font color='darkgreen'>Type Annotations</font>
If you’ve started adopting type hinting, which is now available in Python, then you have another good reason to prefer normal functions over Python lambda functions. Check out [**Python Type Checking** (Guide)]((https://realpython.com/python-type-checking/#hello-types)) to get learn more about Python type hints and type checking. In a lambda function, there is no equivalent for the following:
```python
def full_name(first: str, last: str) -> str:
    return f'{first.title()} {last.title()}'
```
<br/>

Any type error with <font color='blue'>full_name()</font> can be caught by tools like [**mypy**](http://mypy-lang.org/) or [**pyre**](https://pyre-check.org/), whereas a [**SyntaxError**](https://realpython.com/invalid-syntax-python/) with the equivalent lambda function is raised at runtime:
```python
>>> lambda first: str, last: str: first.title() + " " + last.title() -> str
  File "<stdin>", line 1
    lambda first: str, last: str: first.title() + " " + last.title() -> str

SyntaxError: invalid syntax
```
<br/>

Like trying to include a statement in a lambda, adding type annotation immediately results in a [**SyntaxError**](https://realpython.com/invalid-syntax-python/) at runtime.

<a id='sect2_4'></a>
### <font color='darkgreen'>IIFE</font>
You’ve already seen several examples of [immediately invoked function execution](https://developer.mozilla.org/en-US/docs/Glossary/IIFE):
```python
>>> (lambda x: x * x)(3)
9
```
<br/>

<b>It’s a direct consequence of a lambda function being callable as it is defined</b>. For example, this allows you to pass the definition of a Python lambda expression to a higher-order function like [map()](https://docs.python.org/3/library/functions.html#map), [filter()](https://docs.python.org/3/library/functions.html#filter), or [**functools**.reduce()](https://docs.python.org/3/library/functools.html#functools.reduce), or to a `key function`.

<a id='sect3'></a>
## <font color='darkblue'>常常和 Lambda 一起使用的函數 map/filter/functools.reduce 與 key functions</font> ([back](#sect0))
* <font size='3ptx'><b><a href='#sect3_1'>Built-in map/filter & functools.reduce</a></b></font>
* <font size='3ptx'><b><a href='#sect3_2'>key functions 淺談</a></b></font>

<a id='sect3_1'></a>
### <font color='darkgreen'>Built-in map/filter & functools.reduce</font>
[map](https://docs.python.org/3/library/functions.html#map) 與 [filter](https://docs.python.org/3/library/functions.html#filter) 是 Python 預設就支援的函數. [reduce](https://docs.python.org/3/library/functools.html#functools.reduce) 則必須到 [**functools**](https://docs.python.org/3/library/functools.html) 套件下去 import. 底下是這三個函數使用的示意圖:
[image source](https://www.reddit.com/r/ProgrammerHumor/comments/55ompo/map_filter_reduce_explained_with_emojis/)
![map/filter/reduce usage](images/3.PNG)
<br/>

對我來說 [reduce](https://docs.python.org/3/library/functools.html#functools.reduce) 更像是:
![map/filter/reduce usage](images/4.PNG)
<br/>

來看幾個例子吧:

In [3]:
class Beef:
    def __init__(self):
        self.is_veg = False
        
    def cook(self): return 'Hamburger'
    
    
class Potato:
    def __init__(self):
        self.is_veg = True
    
    def cook(self): return 'French Fries'

    
class Chicken:
    def __init__(self):
        self.is_veg = False
        
    def cook(self): return 'Fried chicken'
    
    
class Corn:
    def __init__(self):
        self.is_veg = True
        
    def cook(self): return 'Popcorn'

    
food_ingredients = [Beef(), Potato(), Chicken(), Corn()]

#### <font size='3ptx'>map 範例</font>
[map](https://docs.python.org/3/library/functions.html#map) 需要你提供一個 function 與一個 iterable 物件 (延伸閱讀: [`The Iterator Protocol`](https://www.pythonmorsels.com/iterator-protocol/)), 接著 map 會將 iterable 的每個 element 丟掉你提供的 function 並收集 return 結果並回傳另一個 iterable 物件給你. 底下我們的範例:
* **function**: `lambda food_ingredient: food_ingredient.cook()`
* **iterable 物件**: `food_ingredients`

In [12]:
# map(function, iterable, ...): 
#     Return an iterator that applies function to every item of iterable, yielding the results.
map_iter = map(lambda food_ingredient: food_ingredient.cook(), food_ingredients)
isinstance(map_iter, Iterable)  # map_iter is an iterable object.

True

In [13]:
list(map_iter)

['Hamburger', 'French Fries', 'Fried chicken', 'Popcorn']

#### <font size='3ptx'>filter 範例</font>
[filter](https://docs.python.org/3/library/functions.html#filter) 函數透過你提供的 function 來選擇傳入 iterable 物件中的 element (element 傳進 function 得到 True 的會被選擇).

In [20]:
# filter(function, iterable):
#    Construct an iterator from those elements of iterable for which function returns true.
veg_iter = filter(lambda food_ingredient: food_ingredient.is_veg, food_ingredients)
isinstance(veg_iter, Iterable)  # veg_iter is an iterable object.

True

In [21]:
# 只有 Proato 與 Corn 被選擇, 因為他們 `is_veg` == True
list(veg_iter)

[<__main__.Potato at 0x7f61700a84c0>, <__main__.Corn at 0x7f61700a82e0>]

#### <font size='3ptx'>reduce 範例</font>
reduce 函數的用法用講的不好說, 直接看範例:

In [22]:
from functools import reduce

In [23]:
# If initializer is not given, the first item of iterable object is returned.
f = lambda a, b: a+b
reduce(
    f, 
    [1, 2, 3, 4, 5]
)

15

上面的執行過程可以看成:
![reduce flow](https://nbviewer.org/github/johnklee/oo_dp_lesson/blob/master/lessons/Test_and_function_programming_in_Python/images/fp_6.PNG)
<br/>

事實上你可以提供初始值, 例如:

In [26]:
reduce(
    lambda a, b: a+b, 
    [1, 2, 3, 4, 5],
    10,
)

25

更多有關這個函數的用法, 可以參考 [**Python's reduce(): From Functional to Pythonic Style**](https://realpython.com/python-reduce-function/)

<a id='sect3_2'></a>
### <font color='darkgreen'>key functions 淺談</font>
<b><font size='3ptx'>Key functions in Python are higher-order functions that take a parameter `key` as a named argument.</font></b>

在 Python 許多的函數有提供參數 `key` 便是 lambda 使用的場合之一, 例如:
* [sort()](https://docs.python.org/3/library/stdtypes.html#list.sort): list method
* [sorted()](https://docs.python.org/3/library/functions.html#staticmethod), [min()](https://docs.python.org/3/library/functions.html#min), [max()](https://docs.python.org/3/library/functions.html#max): built-in functions
* [nlargest()](https://docs.python.org/3/library/heapq.html#heapq.nlargest) and [nsmallest()](https://docs.python.org/3/library/heapq.html#heapq.nsmallest): in the Heap queue algorithm module [**heapq**](https://docs.python.org/3/library/heapq.html)
<br/>

來看幾個範例來理解用法.

#### sorted
考慮你有如下的 list:

In [29]:
ids = ['id1', 'id2', 'id100', 'id30', 'id3', 'id22']

你希望透過 `id<num>` 的 `<num>` 來進行排序 (ascending), 這時 [sorted](https://docs.python.org/3/library/functions.html#sorted) 便可以派上用場:
* **sorted(iterable, /, *, <font color='red'>key=None</font>, reverse=False)**: Return a new sorted list from the items in iterable.

In [31]:
sorted(
    ids,
    key=lambda id_str: int(id_str[2:]),  # 比對時使用的值
)

['id1', 'id2', 'id3', 'id22', 'id30', 'id100']

懂一個 Key function 的用法, 其他就是依此類推了, 例如取出最大 `<num>` 的 id 就會是:

In [32]:
max(
    ids,
    key=lambda id_str: int(id_str[2:]))

'id100'

<a id='sect4'></a>
## <font color='darkblue'>可以接受的 Lambda 使用時機</font> ([back](#sect0))
底下是 readability 文件對 Lambda 使用的建議:

* [**2.10 Lambda Functions**](https://engdoc.corp.google.com/eng/doc/devguide/py/style/index.md?cl=head#lambdas)
> Okay to use them for one-liners. If the code inside the lambda function is longer than 60-80 chars, it's probably better to define it as a regular [nested function](https://engdoc.corp.google.com/eng/doc/devguide/py/style/index.md?cl=head#lexical-scoping). <br/><br/>
> For common operations like multiplication, use the functions from the operator module instead of lambda functions. For example, prefer [**operator**.mul](https://docs.python.org/3/library/operator.html#operator.mul) to `lambda x, y: x * y`.

### <font color='darkgreen'>Alternatives to Lambdas</font>
個人在 readability review 不會特別 high light lambda 的使用, 但是有收到一些 review comment 建議可以使用其他方式來取代 lambda 用法. 這邊來看幾個範例.

#### <font size='3ptx'>Map</font>
**The built-in function [map()](https://docs.python.org/3/library/functions.html#map) takes a function as a first argument and applies it to each of the elements of its second argument, an iterable**. Examples of iterables are strings, lists, and tuples. For more information on iterables and iterators, check out [**Iterables and Iterators**](https://realpython.com/lessons/looping-over-iterables/).

[map()](https://docs.python.org/3/library/functions.html#map) returns an iterator corresponding to the transformed collection. As an example, if you wanted to transform a list of strings to a new list with each string capitalized, you could use [map()](https://docs.python.org/3/library/functions.html#map), as follows:

In [33]:
# Map example
list(map(lambda x: x.capitalize(), ['cat', 'dog', 'cow']))

['Cat', 'Dog', 'Cow']

In [34]:
# Proposed way in using list comprehension
[w.capitalize() for w in ['cat', 'dog', 'cow']]

['Cat', 'Dog', 'Cow']

#### <font size='3ptx'>Filter</font>
The built-in function [filter()](https://docs.python.org/3/library/functions.html#filter), another classic functional construct, can be converted into a list comprehension. It takes a [predicate](https://en.wikipedia.org/wiki/Predicate_(mathematical_logic)) as a first argument and an iterable as a second argument. It builds an iterator containing all the elements of the initial collection that satisfies the predicate function. Here’s an example that filters all the even numbers in a given list of integers:

In [35]:
# Filter example
even = lambda x: x%2 == 0
list(filter(even, range(11)))

[0, 2, 4, 6, 8, 10]

In [36]:
# Proposed way in using list comprehension
[x for x in range(11) if x%2 == 0]

[0, 2, 4, 6, 8, 10]

#### <font size='3ptx'>Reduce</font>
Since Python 3, [reduce()](https://docs.python.org/3/library/functools.html#functools.reduce) has gone from a built-in function to a [**functools**](https://docs.python.org/3/library/functools.html#functools.reduce) module function. As [map()](https://docs.python.org/3/library/functions.html#map) and [filter()](https://docs.python.org/3/library/functions.html#filter), its first two arguments are respectively a function and an iterable. It may also take an initializer as a third argument that is used as the initial value of the resulting accumulator. For each element of the iterable, [reduce()](https://docs.python.org/3/library/functools.html#functools.reduce) applies the function and accumulates the result that is returned when the iterable is exhausted.

To apply [reduce()](https://docs.python.org/3/library/functools.html#functools.reduce) to a list of pairs and calculate the sum of the first item of each pair, you could write this:
```python
>>> import functools
>>> pairs = [(1, 'a'), (2, 'b'), (3, 'c')]
>>> functools.reduce(lambda acc, pair: acc + pair[0], pairs, 0)
6
```
<br/>

A more idiomatic approach using a [generator expression](https://www.python.org/dev/peps/pep-0289/), as an argument to [sum()](https://docs.python.org/3/library/functions.html#sum) in the example, is the following:

In [37]:
pairs = [(1, 'a'), (2, 'b'), (3, 'c')]
sum(x[0] for x in pairs)

6

In [38]:
generator = (x[0] for x in pairs)
generator

<generator object <genexpr> at 0x7f61518784a0>

In [42]:
iterator = pairs.__iter__()
iterator

有關 generator 與 iterator 的介紹與說明, 可以參考 "[**How to Use Generators and yield in Python**](https://realpython.com/introduction-to-python-generators/)" 與 "[**The Python `for` loop**](https://realpython.com/python-for-loop/#the-python-for-loop)"

<a id='sect5'></a>
## <font color='darkblue'>Review 時會被打槍的用法</font> ([back](#sect0))
* <font size='3ptx'><b><a href='#sect5_1'>g-long-lambda</a></b></font>
* <font size='3ptx'><b><a href='#sect5_2'>unnecessary-lambda</a></b></font>
<br/>

**<font color='darkred'>The next sections illustrate a few examples of lambda usages that should be avoided</font>**. Those examples might be situations where, in the context of Python lambda, the code exhibits the following pattern:
* It doesn’t follow the Python style guide ([PEP 8](https://peps.python.org/pep-0008/))
* It’s cumbersome and difficult to read.
* It’s unnecessarily clever at the cost of difficult readability.

<a id='sect5_1'></a>
### <font color='darkgreen'>g-long-lambda</font>
> ([link](go/gpylint-faq#g-long-lambda)) Used when a tricky functional-programming construct may be too long.

* <b><font color='darkred'>Negative:</font></b>
```python
users = [
    {'name': 'John', 'age': 40, 'sex': 1}, 
    {'name': 'Ken', 'age': 26, 'sex': 0},
    ...
]
sorted_users = sorted(
    users,
    key=lambda u: (u['age'], u['sex']) if is_employee(u) else (u['age'], u['name']),
)
```

<a id='sect5_2'></a>
### <font color='darkgreen'>unnecessary-lambda</font>
> ([link](go/gpylint-faq#unnecessary-lambda)) Lambda may not be necessary

* <b><font color='darkred'>Negative:</font></b>
```python
foo = {'x': 1, 'y': 2}
self.mock_fn_that_returns_dict = lambda: foo.copy()
```

* <b><font color='green'>Example:</font></b>
```python
foo = {'x': 1, 'y': 2}
self.mock_fn_that_returns_dict = foo.copy
```

<a id='sect6'></a>
## <font color='darkblue'>FPU 簡介</font> ([back](#sect0))
* <font size='3ptx'><b><a href='#sect6_1'>functional composition</a></b></font>
* <font size='3ptx'><b><a href='#sect6_2'>Built-in filter/map/reduce in collection object</a></b></font>
<br/>

[**fpu**](https://github.com/johnklee/fpu) (<font color='brown'>Functional programming utility</font>) 是我維護的一個 Python package 用來提升 Python 對 FP 的支援. 這邊帶幾個範例來了解它帶來的好處.

<a id='sect6_1'></a>
### <font color='darkgreen'>functional composition</font>
Functional composition 的特性 (延伸閱讀: "[**Function composition and lazy execution**](https://ithelp.ithome.com.tw/articles/10235556)") 讓你可以方便的串接函數來產生新的函數. 考慮你有以下代碼:

In [44]:
data_set = [{'values':[1, 2, 3]}, {'values':[4, 5]}]

# Imperative
def min_max_imp(data_set):
    """Picks up the maximum of each element and calculate the minimum of them."""
    max_list = []
    for d in data_set:
        max_list.append(max(d['values']))
        
    return min(max_list)

In [46]:
# Max of [1, 2, 3] -> [3], max of [4, 5] -> [5] => Got [3, 5]
# Min of [3, 5] => 3
min_max_imp(data_set)

3

事實上這是兩個函數 min/max 串接的結果. 透過 FPU, 你可以改寫成:

In [47]:
# FP
from fpu.fp import *
from functools import reduce, partial

# compose2(f, g) = f(g())
min_max = compose2(
    partial(reduce, min),   # [3, 5] -> [3]
    partial(map, lambda d: max(d['values'])))  # [{'values':[1, 2, 3]}, {'values':[4, 5]}] -> [3, 5]

min_max(data_set)

3

<a id='sect6_2'></a>
### <font color='darkgreen'>Built-in filter/map/reduce in collection object</font>
FPU 中的 collection 物件自帶 filter/map/reduce 函數. 考慮下面問題:

In [49]:
# 請找出在每個 element 都有出現過的元素 (character).
arr = ['abcdde', 'baccd', 'eeabg']

In [52]:
def gemstones_imp(arr):
    # 1) Collect unique character of each element
    set_list = []
    for s in arr:
        set_list.append(set(list(s)))
        
    # 2) Keep searching overlapping characters among all set
    uset = set_list[0]
    for aset in set_list[1:]:
        uset = uset & aset
        
    return ''.join(uset)

In [53]:
gemstones_imp(arr)

'ab'

使用 FPU 改寫變成:

In [56]:
from fpu.flist import *

def gemstones_dec(arr):
    rlist = fl(arr)
    return ''.join(
        rlist.map(
            # 將每個 element 轉成 set
            lambda e: set(list(e))
        ).reduce(
            # 依序找出每個 set 共用的 character
            lambda a, b: a & b
        )
    )

In [55]:
gemstones_dec(arr)

'ab'

## <font color='darkblue'>Supplement</font>
* [Medium - 聊聊 Python Closure](https://dboyliao.medium.com/%E8%81%8A%E8%81%8A-python-closure-ebd63ff0146f)
* [FPU - Functional programming utility (slides)](https://docs.google.com/presentation/d/1e8JkC1253jmfWIwppDbWFpy51m4P-uM0LZxqpPQCUjs/edit?usp=sharing&resourcekey=0-krY5MI7h9oGfveN4D8AN_w)
* [Introduction of FP in Python (notebook)](https://nbviewer.org/github/johnklee/oo_dp_lesson/blob/master/lessons/Test_and_function_programming_in_Python/py_fp.ipynb)